In [4]:
import os
import time
import openpyxl
import locale
import pandas as pd
from loguru import logger
from datetime import datetime
from dotenv import load_dotenv
from sqlalchemy import create_engine, exc

load_dotenv()

True

In [5]:
def get_db_engine():
    try:
        db_url = os.getenv('DUSNEI_URL')
        engine = create_engine(db_url)
        # Test connection
        with engine.connect() as connection:
            print('Conexão estabelecida!')
            pass
        print('Banco de dados conectado!')
        return engine
    except exc.SQLAlchemyError as e:
        print(f"Error: {e}")
        return None
        


In [ ]:
def supervisores(conn):
    cod_supervisores = []

    query = (f"""
        SELECT
            supe.super_cod AS supe_cod,
            supe.supe_nome AS supe_nome
        FROM pendfin AS pfin
        INNER JOIN planoger AS pger ON pfin.pfin_pger_conta = pger.pger_conta
        INNER JOIN unidades AS unid ON pfin.pfin_unid_codigo = unid.unid_codigo
        INNER JOIN clientes AS clie ON pfin.pfin_codentidade = clie.clie_codigo
        INNER JOIN vendedores AS vend ON clie.clie_vend_codigo = vend.vend_codigo
        INNER JOIN supervisores AS supe ON vend.vend_supe_codigo = supe.supe_codigo
        WHERE pfin.pfin_status = 'P'
        AND pfin.pfin_datavcto < CURRENT_DATE
        AND vend.vend_supe_codigo IN {tuple(cod_supe)}
        ORDER BY pfin.pfin_datavcto DESC
    """)
    df = pd.read_sql_query(query, conn)

    # Adiciona os códigos dos supervisores a lista, evitando duplicados
    for supe_cod in df['supe_cod'].unique():
        if supe_cod not in supervisores:
            supervisores.append(supe_cod)

    return df

In [7]:
path_dados = os.getenv('DUSNEI_DATA_DIRECTORY_RELATORIO_PENDENCIAS')
conn = get_db_engine()
supervisores = []
        
def pendencias_clientes_query(conn, cod_supe):
    query = (f"""
        SELECT
            pfin.pfin_transacao AS transacao,
            pfin.pfin_status AS status,
            pfin.pfin_datavcto AS datavcto,
            pfin.pfin_pger_conta AS pger_conta,
            pfin.pfin_unid_codigo AS unid_cod,
            pfin.pfin_vend_codigo AS vend_cod,
            pfin.pfin_codentidade AS entidade_cod,
            pfin.pfin_cnpjcpf AS cnpj,
            pfin.pfin_numerodcto AS nota,
            pfin.pfin_parcela AS parcelas,
            pfin.pfin_valor AS valor,
            pfin.pfin_nparcelas AS nparcelas,
            pger.pger_descricao AS descricao_contabil,
            pger.pger_tipo AS tipo,
            clie.clie_nome AS clie_nome,
            clie.clie_razaosocial AS razaosocial,
            clie.clie_vend_codigo AS clie_vend_cod,
            clie.clie_foneres AS fone_res,
            clie.clie_fonecel AS fone_cel,
            vend.vend_nome AS vend_nome,
            vend.vend_supe_codigo AS supe_cod,
            CONCAT(vend.vend_extra16, vend.vend_extra15) AS vend_cel,
            unid.unid_reduzido AS unidade,
            supe.supe_nome AS supe_nome
        FROM pendfin AS pfin
        INNER JOIN planoger AS pger ON pfin.pfin_pger_conta = pger.pger_conta
        INNER JOIN unidades AS unid ON pfin.pfin_unid_codigo = unid.unid_codigo
        INNER JOIN clientes AS clie ON pfin.pfin_codentidade = clie.clie_codigo
        INNER JOIN vendedores AS vend ON clie.clie_vend_codigo = vend.vend_codigo
        INNER JOIN supervisores AS supe ON vend.vend_supe_codigo = supe.supe_codigo
        WHERE pfin.pfin_status = 'P'
        AND pfin.pfin_datavcto < CURRENT_DATE
        AND vend.vend_supe_codigo IN {tuple(cod_supe)}
        ORDER BY pfin.pfin_datavcto DESC
    """)
    df = pd.read_sql_query(query, conn)

    # Adiciona os códigos dos supervisores a lista, evitando duplicados
    for supe_cod in df['supe_cod'].unique():
        if supe_cod not in supervisores:
            supervisores.append(supe_cod)

    return df

def save_to_excel(cod_supe):
    locale.setlocale(locale.LC_MONETARY, 'pt-BR.UTF-8')
    df = pendencias_clientes_query(conn, cod_supe)
    
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = f"Supervisor {cod_supe}"
    
    columns_title = [
        'Index','CodClie','RazaoSocial','CNPJ','Nota','Valor','Vencimento','Fone','Celular','CodSupe','Supervisor','CodVend','Vendedor','Celular','C.Descrição','Status','Hora'
    ]
    row_title = 1

    for col_num, column_title in enumerate(columns_title, 1):
        cell = ws.cell(row=row_title, column=col_num)
        cell.value = column_title

    for index, row in enumerate(df.itertuples(), start=2):
        status = "-"
        hora = "-"
        index = 1
        
        ws.cell(row=index, column=1).value = index + 1
        ws.cell(row=index, column=2).value = row.entidade_cod
        ws.cell(row=index, column=3).value = row.razaosocial
        ws.cell(row=index, column=4).value = row.cnpj
        ws.cell(row=index, column=5).value = row.nota
        ws.cell(row=index, column=6).value = row.valor
        ws.cell(row=index, column=7).value = format(row.datavcto, "%Y/%m/%d")
        ws.cell(row=index, column=8).value = row.fone_res
        ws.cell(row=index, column=9).value = row.fone_cel
        ws.cell(row=index, column=10).value = row.supe_cod
        ws.cell(row=index, column=11).value = row.supe_nome
        ws.cell(row=index, column=12).value = row.clie_vend_cod
        ws.cell(row=index, column=13).value = row.vend_nome
        ws.cell(row=index, column=14).value = row.vend_cel
        ws.cell(row=index, column=15).value = row.descricao_contabil
        ws.cell(row=index, column=16).value = status
        ws.cell(row=index, column=17).value = hora

        data_pasta = datetime.now().strftime("%Y-%m-%d")
        nome_arquivo = (f'pendencias-{cod_supe}-{data_pasta}')
        diretorio = f'{path_dados}/{data_pasta}'
        if not os.path.exists(diretorio):
            os.mkdir(diretorio)
        local_arquivo = os.path.join(f'{diretorio}/{nome_arquivo}.xlsx')
        wb.save(local_arquivo)


Conexão estabelecida!
Banco de dados conectado!


In [9]:
def excel_for_supe():
    for supervisor in supervisores:
        df = pendencias_clientes_query(conn, supervisor)
        save_to_excel(supervisor)

excel_for_supe()